# Spark Apache (семинары)

## Урок 5. Spark on scala

Условие: создайте csv файл с таким содержимым:

```
title,author,genre,sales,year
"1984", "George Orwell", "Science Fiction", 5000, 1949
"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954
"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960
"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951
"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925
```

Задание:

- Используя Spark прочитайте данные из файла csv.

- Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.

- Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.

- Отсортируйте данные по общему объему продаж в порядке убывания.

- Выведите результаты на экран.


In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

# Создание Spark сессии
spark = SparkSession.builder.appName("Homework5").getOrCreate()

# Загрузка данных
df = spark.read.option("header",True).csv("books.csv")
print("Исходный датасет:")
df.show()

# Фильтрация данных, чтобы оставить только книги, продажи которых превышают 3000 экземпляров
filtered_df = df.filter(df.sales > 3000)
print("Фильтрация по продажам (более 3000 экземпляров):")
filtered_df.show()

# Группировка данных по жанру и вычисление общего объема продаж для каждого жанра
window_spec = Window.partitionBy("genre") 
sum_sales_by_genre = fn.sum("sales").over(window_spec)
sum_sales_by_genre_df = df.withColumn("sum_sales_by_genre", sum_sales_by_genre)
# Сортировка данных по общему объему продаж в порядке убывания и вывод результатов:
print("Общие объемы продаж по жанрам в порядке убывания:")
sum_sales_by_genre_df.select(["genre", "sum_sales_by_genre"]).distinct() \
    .orderBy(sum_sales_by_genre_df.sum_sales_by_genre.desc()).show()

Исходный датасет:
+--------------------+--------------------+------------------+-----+-----+
|               title|              author|             genre|sales| year|
+--------------------+--------------------+------------------+-----+-----+
|                1984|     "George Orwell"| "Science Fiction"| 5000| 1949|
|The Lord of the R...|    "J.R.R. Tolkien"|         "Fantasy"| 3000| 1954|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"| 4000| 1960|
|The Catcher in th...|     "J.D. Salinger"|           "Novel"| 2000| 1951|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"| 4500| 1925|
+--------------------+--------------------+------------------+-----+-----+

Фильтрация по продажам (более 3000 экземпляров):
+--------------------+--------------------+------------------+-----+-----+
|               title|              author|             genre|sales| year|
+--------------------+--------------------+------------------+-----+-----+
|                1984|     "Geor